
# Maize Fall Armyworm — Baseline (PyTorch, ResNet18, AUC)

Binary image classification to detect Fall Armyworm on maize leaves.  
This notebook is **Colab-ready** (works on Tesla K80) and conforms to the hackathon rules.


In [ ]:

# If running on Colab, uncomment the next line to ensure sklearn is present (usually already installed).
# !pip -q install -U scikit-learn


## Quick EDA: Class balance & Samples

In [ ]:

import matplotlib.pyplot as plt

counts = train_df['Label'].value_counts().sort_index()
print(counts)
plt.figure()
counts.plot(kind='bar')
plt.title('Class counts (0=Healthy, 1=Fall Armyworm)')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()


In [ ]:

# Show a few random training images with labels
from math import ceil
import matplotlib.pyplot as plt

sample_df = train_df.sample(9, random_state=SEED).reset_index(drop=True)
fig = plt.figure(figsize=(9,9))
for i, row in sample_df.iterrows():
    ax = plt.subplot(3,3,i+1)
    img_path = resolve_image_path(row['Image_id'], IMAGES_DIR)
    im = Image.open(img_path).convert('RGB')
    plt.imshow(im)
    ax.set_title(f"{row['Label']} - {Path(img_path).name}", fontsize=8)
    plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:

import os, sys, math, time, random, shutil, glob, json
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from tqdm.auto import tqdm

SEED = 1337
def seed_everything(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()
print('PyTorch', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device



## Paths

Set the paths below. If you're in Colab:
- Upload/Unzip `Images.zip` into the working directory so that images live under `Images/`
- Upload the three CSVs (`Train.csv`, `Test.csv`, `SampleSubmission.csv`) to the working directory.


In [ ]:

# ==== CONFIG ====
IMAGES_DIR = Path('Images')   # folder containing all images
TRAIN_CSV = Path('Train.csv')
TEST_CSV  = Path('Test.csv')
SAMPLE_SUB = Path('SampleSubmission.csv')

IMG_EXTS = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']

assert TRAIN_CSV.exists() and TEST_CSV.exists(), "Make sure Train.csv and Test.csv are present."
train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)

print(train_df.head(), '\n', test_df.head())
print('Train size:', len(train_df), ' Test size:', len(test_df))


In [ ]:

def resolve_image_path(img_id, images_dir=IMAGES_DIR):
    # Try common extensions. The CSV appears to include IDs without extension.
    for ext in IMG_EXTS:
        p = images_dir / f"{img_id}{ext}"
        if p.exists():
            return p
    # Sometimes the ID already includes an extension:
    p = images_dir / img_id
    if p.exists():
        return p
    raise FileNotFoundError(f"Could not find image for id={img_id} under {images_dir}")


In [ ]:

class MaizeDataset(Dataset):
    def __init__(self, df, images_dir, mode='train', transform=None):
        self.df = df.reset_index(drop=True).copy()
        self.images_dir = Path(images_dir)
        self.mode = mode
        self.transform = transform
        
        if self.mode != 'test':
            assert 'Label' in self.df.columns, "Train/val dataframe must have a 'Label' column."

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_id = row['Image_id']
        img_path = resolve_image_path(img_id, self.images_dir)

        img = Image.open(img_path).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        
        if self.mode == 'test':
            return img, img_id
        else:
            label = float(row['Label'])
            label = torch.tensor(label, dtype=torch.float32)
            return img, label


In [ ]:

IMG_SIZE = 224  # Fits K80 easily; can try 256/320 later
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

valid_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE+32),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:

train_split, valid_split = train_test_split(
    train_df, test_size=0.2, stratify=train_df['Label'], random_state=SEED
)
print('Split sizes:', len(train_split), len(valid_split))


In [ ]:

BATCH_SIZE = 32   # for K80; adjust to 64 if memory allows
NUM_WORKERS = 2

train_ds = MaizeDataset(train_split, IMAGES_DIR, mode='train', transform=train_tfms)
valid_ds = MaizeDataset(valid_split, IMAGES_DIR, mode='train', transform=valid_tfms)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

len(train_loader), len(valid_loader)


In [ ]:

def build_model():
    # ResNet18 is a strong/light baseline
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    # Replace head
    in_feats = model.fc.in_features
    model.fc = nn.Linear(in_feats, 1)
    return model

model = build_model().to(device)
sum(p.numel() for p in model.parameters())/1e6


In [ ]:

class EarlyStopper:
    def __init__(self, patience=3, mode='max', min_delta=1e-4):
        self.patience = patience
        self.mode = mode
        self.min_delta = min_delta
        self.best = -np.inf if mode=='max' else np.inf
        self.wait = 0

    def step(self, value):
        improved = (value > self.best + self.min_delta) if self.mode=='max' else (value < self.best - self.min_delta)
        if improved:
            self.best = value
            self.wait = 0
            return True  # signal to save
        else:
            self.wait += 1
            return False  # no save

    def should_stop(self):
        return self.wait >= self.patience


In [ ]:

def train_one_epoch(model, loader, optimizer, scaler):
    model.train()
    running_loss = 0.0
    for (imgs, labels) in tqdm(loader, leave=False):
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True).unsqueeze(1)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            logits = model(imgs)
            loss = F.binary_cross_entropy_with_logits(logits, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item() * imgs.size(0)
    return running_loss / len(loader.dataset)

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    all_probs, all_targets = [], []
    for (imgs, labels) in tqdm(loader, leave=False):
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True).unsqueeze(1)
        logits = model(imgs)
        probs = torch.sigmoid(logits)
        all_probs.append(probs.detach().cpu().numpy())
        all_targets.append(labels.detach().cpu().numpy())
    all_probs = np.vstack(all_probs).ravel()
    all_targets = np.vstack(all_targets).ravel()
    auc = roc_auc_score(all_targets, all_probs)
    return auc, all_probs, all_targets


In [ ]:

EPOCHS = 10
LR = 3e-4

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
early = EarlyStopper(patience=3, mode='max')

best_auc = -1
best_path = 'best_resnet18.pt'

for epoch in range(1, EPOCHS+1):
    train_loss = train_one_epoch(model, train_loader, optimizer, scaler)
    val_auc, _, _ = evaluate(model, valid_loader)
    print(f"Epoch {epoch:02d}: train_loss={train_loss:.4f}  val_auc={val_auc:.4f}")
    if early.step(val_auc):
        torch.save({'model': model.state_dict(), 'auc': val_auc}, best_path)
        print(f"  ✓ Saved new best model (AUC={val_auc:.4f})")
    if early.should_stop():
        print("Early stopping.")
        break

print('Best AUC recorded:', early.best)


In [ ]:

# Reload best
ckpt = torch.load('best_resnet18.pt', map_location=device)
model.load_state_dict(ckpt['model'])
model.eval()

test_ds = MaizeDataset(test_df, IMAGES_DIR, mode='test', transform=valid_tfms)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

probs_list, ids = [], []
@torch.no_grad()
def predict_tta(imgs, model):
    # simple 2x TTA: original + hflip
    logits = model(imgs)
    logits_flip = model(torch.flip(imgs, dims=[3]))  # horizontal flip
    probs = torch.sigmoid((logits + logits_flip) / 2.0)
    return probs

for (imgs, img_ids) in tqdm(test_loader):
    imgs = imgs.to(device, non_blocking=True)
    probs = predict_tta(imgs, model)
    probs_list.append(probs.detach().cpu().numpy().ravel())
    ids.extend(img_ids)

probs = np.concatenate(probs_list)
sub = pd.DataFrame({'Image_id': ids, 'Label': probs})
sub.to_csv('submission.csv', index=False)
print(sub.head())
print('Wrote submission.csv')



## Tips & Next Steps
- Try larger input (`IMG_SIZE=256`/`320`) if time allows.
- Increase **epochs** to ~15–20 with early stopping.
- Try **ResNet34** or **EfficientNet** (if allowed through `torchvision` or `timm`).
- Use **StratifiedKFold** (3–5 folds) and **average predictions** (ensembling) within time limits.
- Add **MixUp/CutMix**, or stronger augmentations.
- Use **label smoothing** (e.g., BCE with smoothing) to regularize.
- Add **Test-Time Augmentation (TTA)** with more flips/rotations (respecting inference time caps).
- Always submit **probabilities** (not rounded values) to optimize AUC.


In [ ]:

# Optional: Export to TorchScript (for mobile/edge)
example = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(device)
traced = torch.jit.trace(model, example)
ts_path = "resnet18_faw.torchscript.pt"
traced.save(ts_path)
print("Saved TorchScript model to", ts_path)
